In [5]:
import os # need for filepath stuff
import sys # need for command line stuff
import random # for reasons
import math

pathstr = 'C:\\Users\\Sky\\Documents\\Spam or Ham\\train\\4'

hamFiles = []
spamFiles = []

numHamWords = 0
numSpamWords = 0

# this part takes forever unfortunately

numTxtFiles = 0
numHamFiles = 0
numSpamFiles = 0

for root, dir, files in os.walk(pathstr):
    #print(dir)
    for file in files:
        if '.txt' in file:
            numTxtFiles = numTxtFiles + 1
            #print('Text file #' + str(numTxtFiles))
            if '.ham' in file:
                numHamFiles += 1
                #print('Ham file #' + str(numHamFiles))
                hamFiles.append(os.path.join(root, file))
            if '.spam' in file:
                numSpamFiles += 1
                #print('Spam file #' + str(numSpamFiles))
                spamFiles.append(os.path.join(root, file))

                
### -----HAM WORDS----- ###

hamContentFreq = {}

for hf in hamFiles:
    currFile = open(hf, 'r', encoding='latin1') # open file
    currContent = currFile.read()
    currContent = currContent.split() # hamContent becomes a list of strings
    numHamWords += len(currContent)
    for x in currContent: # for every item in list of strings
        if x in hamContentFreq.keys(): # if already in dictionary, increment frequency in dictionary
            hamContentFreq[x] = hamContentFreq[x] + 1
        else: # if not already in dictionary, add to dictionary with frequency 1
            hamContentFreq[x] = 1
    currFile.close()
    #for x in currContent: # for every item in list of strings
    #    if x in hamContentFreq.keys(): # if already in dictionary, increment frequency in dictionary
    #        hamContentFreq[x.lower()] = hamContentFreq[x.lower()] + 1
    #    else: # if not already in dictionary, add to dictionary with frequency 1
    #        hamContentFreq[x.lower()] = 1
    #currFile.close()

#print('---------------HamContentFreq---------------')
#print(hamContentFreq)

### -----SPAM----- ###

spamContentFreq = {}

for sf in spamFiles:
    currFile = open(sf, 'r', encoding='latin1') # open file
    currContent = currFile.read()
    currContent = currContent.split() 
    numSpamWords += len(currContent)
    for x in currContent: # for every item in list of strings
        if x in spamContentFreq.keys(): # if already in dictionary, increment frequency in dictionary
            spamContentFreq[x] = spamContentFreq[x] + 1
        else: # if not already in dictionary, add to dictionary with frequency 1
            spamContentFreq[x] = 1
    currFile.close()
    #for x in currContent: # for every item in list of strings
    #    if x in spamContentFreq.keys(): # if already in dictionary, increment frequency in dictionary
    #        spamContentFreq[x.lower()] = spamContentFreq[x.lower()] + 1
    #    else: # if not already in dictionary, add to dictionary with frequency 1
    #        spamContentFreq[x.lower()] = 1
    #currFile.close()

#print('---------------SpamContentFreq---------------')
#print(spamContentFreq)


### ---ADD ONE SMOOTHING--- ###

spamFreqNoSmoothing = spamContentFreq
hamFreqNoSmoothing = hamContentFreq

spamFreqSmoothing = spamContentFreq
hamFreqSmoothing = hamContentFreq

hamAOS = [] # list to store all words in spam, but not in ham
spamAOS = [] # list to store all words in ham, but not in spam

# 1: loop through all words in spam database
# 2: if a word encountered in spam doesn't exist in ham, add to hamAOS
# 3: OUTSIDE the loop: add len(hamAOS) to frequency of each word in ham database
# 4: add each word in hamAOS w frequency 1 to ham database

for word in spamFreqNoSmoothing:
    if word not in hamFreqNoSmoothing:
        hamAOS.append(word)

#print(len(hamAOS))

for word in hamFreqNoSmoothing:
    if word not in spamFreqNoSmoothing:
        spamAOS.append(word)
        
#print(len(spamAOS))

#for word in hamFreqNoSmoothing:
#    hamFreqNoSmoothing[word] += len(hamAOS)
for word in hamAOS:
    hamFreqNoSmoothing[word] = 1

#for word in spamFreqNoSmoothing:
#    spamFreqNoSmoothing[word] += len(spamAOS)
for word in spamAOS:
    spamFreqNoSmoothing[word] = 1
    
### -----CALCULATE PROBABILITIES----- ###
spamProbSmoothing = spamFreqSmoothing
for word in spamProbSmoothing:
    #spamProbSmoothing[word] = spamProbSmoothing[word]/len(spamProbSmoothing)
    spamProbSmoothing[word] = math.log(spamProbSmoothing[word]/numSpamWords)


hamProbSmoothing = hamFreqSmoothing    
for word in hamProbSmoothing:
    #hamProbSmoothing[word] = hamProbSmoothing[word]/len(hamProbSmoothing)
    hamProbSmoothing[word] = math.log(hamProbSmoothing[word]/numHamWords)

# output these probabilities to a file
# will need to make this p(ham), p(spam), p(token|ham) and p(token|spam)

outputFileName = 'nbmodel.txt'

outputFile = open(outputFileName, 'w', encoding="latin1")

totalNumFiles = len(hamFiles) + len(spamFiles)

#print("Writing p(spam)...")
# first thing in the output file is p(ham)
#pHam = len(hamFiles)/totalNumFiles
pHam = numHamWords/(numHamWords + numSpamWords)
outputFile.write(str(pHam))
outputFile.write("\n")
         
#print("Writing p(spam)...")
# second thing in the output file is p(spam)
pSpam = numSpamWords/(numHamWords + numSpamWords)
outputFile.write(str(pSpam))
outputFile.write("\n")

#print("Writing p(token|ham)...")
# third is p(token|ham)
for word, prob in hamProbSmoothing.items():
    #print(word + "\t" + str(prob) + "\n")
    outputFile.write(word + "\t" + str(prob) + "\n")
outputFile.write("--------------------\n")

#print("Writing p(token|spam)...")
# fourth is p(token|spam)
for word, prob in spamProbSmoothing.items():
    #print(word + "\t" + str(prob) + "\n")
    outputFile.write(word + "\t" + str(prob) + "\n")
outputFile.write("--------------------\n")

outputFile.close()

In [4]:
import os # need for filepath stuff
# get files from Spam or Ham

pathstr = 'C:\\Users\\Sky\\Documents\\Spam or Ham\\train\\4'

hamFiles = []
spamFiles = []

numHamWords = 0
numSpamWords = 0

# this part takes forever unfortunately

numTxtFiles = 0
numHamFiles = 0
numSpamFiles = 0

for root, dir, files in os.walk(pathstr):
    #print(dir)
    for file in files:
        if '.txt' in file:
            numTxtFiles = numTxtFiles + 1
            #print('Text file #' + str(numTxtFiles))
            if '.ham' in file:
                numHamFiles += 1
                #print('Ham file #' + str(numHamFiles))
                hamFiles.append(os.path.join(root, file))
            if '.spam' in file:
                numSpamFiles += 1
                #print('Spam file #' + str(numSpamFiles))
                spamFiles.append(os.path.join(root, file))
                

In [ ]:
#os.chdir('C:\\Users\\Sky\\Documents\\Spam or Ham\\train\\1\\ham')
#os.getcwd()

### HAM ###

hamContentFreq = {}

for hf in hamFiles:
    currFile = open(hf, 'r', encoding='latin1') # open file
    currContent = currFile.read()
    currContent = currContent.split() # hamContent becomes a list of strings
    numHamWords += len(currContent)
    for x in currContent: # for every item in list of strings
        if x in hamContentFreq.keys(): # if already in dictionary, increment frequency in dictionary
            hamContentFreq[x] = hamContentFreq[x] + 1
        else: # if not already in dictionary, add to dictionary with frequency 1
            hamContentFreq[x] = 1

#print('---------------HamContentFreq---------------')
#print(hamContentFreq)


In [ ]:
### SPAM ###

spamContentFreq = {}

for sf in spamFiles:
    currFile = open(sf, 'r', encoding='latin1') # open file
    currContent = currFile.read()
    currContent = currContent.split() 
    numSpamWords += len(currContent)
    for x in currContent: # for every item in list of strings
        if x in spamContentFreq.keys(): # if already in dictionary, increment frequency in dictionary
            spamContentFreq[x] = spamContentFreq[x] + 1
        else: # if not already in dictionary, add to dictionary with frequency 1
            spamContentFreq[x] = 1

print('---------------SpamContentFreq---------------')
print(spamContentFreq)

In [ ]:
# ADD ONE SMOOTHING
# for any word in HamDict not in SpamDict, add it to SpamDict with frequency of 1
# for any word in SpamDict not in HamDict, add to to HamDict with frequency of 1

spamFreqNoSmoothing = spamContentFreq
hamFreqNoSmoothing = hamContentFreq

spamFreqSmoothing = spamContentFreq
hamFreqSmoothing = hamContentFreq

print(len(spamFreqNoSmoothing))


In [ ]:
#for word in spamFreqNoSmoothing:
#    if word not in hamFreqNoSmoothing:
#        for allOtherWords in hamFreqNoSmoothing:
#            hamFreqSmoothing[allOtherWords] += 1
#        hamFreqSmoothing[word] = 1

hamAOS = [] # list to store all words in spam, but not in ham
spamAOS = [] # list to store all words in ham, but not in spam

# 1: loop through all words in spam database
# 2: if a word encountered in spam doesn't exist in ham, add to hamAOS
# 3: OUTSIDE the loop: add len(hamAOS) to frequency of each word in ham database
# 4: add each word in hamAOS w frequency 1 to ham database

for word in spamFreqNoSmoothing:
    if word not in hamFreqNoSmoothing:
        hamAOS.append(word)

print(len(hamAOS))

for word in hamFreqNoSmoothing:
    if word not in spamFreqNoSmoothing:
        spamAOS.append(word)
        
print(len(spamAOS))

for word in hamFreqNoSmoothing:
    hamFreqNoSmoothing[word] += len(hamAOS)
for word in hamAOS:
    hamFreqNoSmoothing[word] = 1

for word in spamFreqNoSmoothing:
    spamFreqNoSmoothing[word] += len(spamAOS)
for word in spamAOS:
    spamFreqNoSmoothing[word] = 1


In [ ]:
spamProbSmoothing = spamFreqSmoothing
for word in spamProbSmoothing:
    spamProbSmoothing[word] = spamProbSmoothing[word]/len(spamProbSmoothing)

hamProbSmoothing = hamFreqSmoothing    
for word in hamProbSmoothing:
    hamProbSmoothing[word] = hamProbSmoothing[word]/len(hamProbSmoothing)



In [ ]:
# output these probabilities to a file
# will need to make this p(ham), p(spam), p(token|ham) and p(token|spam)

outputFileName = 'nbmodel.txt'

outputFile = open(outputFileName, 'w', encoding="latin1")

totalNumFiles = len(hamFiles) + len(spamFiles)

print("Writing p(spam)...")
# first thing in the output file is p(ham)
#pHam = len(hamFiles)/totalNumFiles
pHam = numHamWords/(numHamWords + numSpamWords)
print(pHam)
outputFile.write(str(pHam))
outputFile.write("\n")
         
print("Writing p(spam)...")
# second thing in the output file is p(spam)
#pSpam = len(spamFiles)/totalNumFiles
pSpam = numSpamWords/(numHamWords + numSpamWords)
print(pSpam)
outputFile.write(str(pSpam))
outputFile.write("\n")

print("Writing p(token|ham)...")
# third is p(token|ham)
for word, prob in hamProbSmoothing.items():
    print(word + "\t" + str(prob) + "\n")
    outputFile.write(word + "\t" + str(prob) + "\n")
outputFile.write("--------------------\n")

print("Writing p(token|spam)...")
# fourth is p(token|spam)
for word, prob in spamProbSmoothing.items():
    print(word + "\t" + str(prob) + "\n")
    outputFile.write(word + "\t" + str(prob) + "\n")
outputFile.write("--------------------\n")

In [ ]:
#for word in hamFreqNoSmoothing:
#    if word not in spamFreqNoSmoothing:
#        for allOtherWords in spamFreqNoSmoothing:
#            spamFreqSmoothing[allOtherWords] += 1
#        spamFreqSmoothing[word] = 1

In [ ]:
#totalHamWords = 0.0 # initialize sum
#hamProbDict = hamContentFreq;

# find sum of words in text
#for x in hamContentFreq:
#    totalHamWords += hamContentFreq[x]

#for x in hamProbDict:
#    hamProbDict[x] = hamProbDict[x]/totalHamWords

        
### SPAM ###        

#totalSpamWords = 0.0 # initialize sum
#spamProbDict = spamContentFreq;

# find sum of words in text
#for x in spamContentFreq:
#    totalSpamWords += spamContentFreq[x]

#for x in spamProbDict:
#    spamProbDict[x] = spamProbDict[x]/totalSpamWords